# Using CNN on Image dataset (CIFAR-10)

#### Authors: Cameron Morefield, Sean Vucinich, Bret Stine
#### Date: 11.9.2018

## Introduction:

This CNN (Convolutional Neural Network) looks to modify the pre-existing code for the MNIST dataset in order to attempt to classify CIFAR Images that are color (3 Channel) and 32 pixels in height width.


In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Required Imports

In [6]:
height = 32
width = 32
channels = 3
n_inputs = height * width * channels

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

tf.reset_default_graph()

Value Initializations of our 32x32 images including the adding of RGB color channels.

In [7]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 8 * 8])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()





Creation and architecture of our neural net. We changed the pool3 to 8 to be a valid subset of the 32x32 data.

In [8]:
# load data
cifar_file = "D:\CST 463\cifar-10-python.tar\cifar-10-batches-py\data_batch_1"

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cf = unpickle(cifar_file)


Our functions for creating batches, as well as converting the CIFAR data to usable python dictionaries.

In [ ]:
# get the data and labels
dat = cf[b'data']
labels = np.array(cf[b'labels'])

scaler = StandardScaler()
dat_scaled = scaler.fit_transform(dat)

X_train, X_test, y_train, y_test = train_test_split(dat_scaled, labels, test_size=0.25)

n_epochs = 10
batch_size = 50

D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Creating our test and training data.

In [ ]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

Execution phase of the CNN - Running on our data.



Conclusion:

Our CNN ended up getting around a 40% This could be improved by modifying various hyper parameters as well as using a using different batches. We did not look into the contents of each of the binary CIFAR files, so we do not know whether there is any form of randomization in the images, which would have a real effect on our accuracy rate. In addition, if the image data is scaled before the execution of the CNN, then the validation accuracy of the neural net improves by roughly 12%.